<a href="https://colab.research.google.com/github/abuelnasr0/Semantic-Search-project/blob/main/Notebooks/vector_database/populate_vdb_with_papers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install grpcio==1.49.1 -q
!pip install pymilvus==2.2.11 -q

In [ ]:
!pip install sentence_transformers -q

In [3]:
uri = ""
token = ""

In [4]:
from pymilvus import connections, Collection

connections.connect("default",
                  uri=uri,
                  token=token)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [6]:
PATH = '/content/drive/MyDrive/Graduation_project/Data/Papers/Arxiv_cleaned/new_data.json'

In [13]:
import json

with open(PATH, encoding='utf-8', mode='r') as json_file:
  loaded_papers = json.load(json_file)

In [14]:
# take only 50 000 of the papers
import random

MAX_NUM_PAPERS = 50_000

papers = loaded_papers
random.shuffle(papers)
papers = papers[0:MAX_NUM_PAPERS]

In [ ]:
max_title_chars = 0
max_abstract_chars = 0

for paper in papers:
  max_abstract_chars = max(max_abstract_chars, len(paper['abstract']))
  max_title_chars = max(max_title_chars, len(paper['title']))

print("max title chars = ", max_title_chars)
print("max abstract chars = ", max_abstract_chars)

In [ ]:
max_id_chars = 0

for paper in papers:
  max_id_chars = max(max_id_chars, len(paper['id']))

print("max id chars = ", max_id_chars)

In [17]:
papers_ids = []
titles = []
abstracts = []

for paper in papers:
  papers_ids.append(paper['id'])
  titles.append(paper['title'])
  abstracts.append(paper['abstract'])

In [ ]:
from sentence_transformers import SentenceTransformer
import torch
import os
model_name = "/content/drive/MyDrive/Graduation_project/Models/msmarco-bert-base-dot-v5-v2-Titles-wiht_150000_samples"

# model_name = os.getenv('MODEL_NAME', 'msmarco-bert-base-dot-v5')
assert model_name, "The model name is not avilable"

model = SentenceTransformer(model_name)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)
print('After Checking Cude: {}'.format(model.device))

In [19]:
embeddings = model.encode(abstracts, batch_size=256)

Batches:   0%|          | 0/196 [00:00<?, ?it/s]

In [ ]:
embeddings = embeddings.tolist()
len(embeddings)

In [ ]:
NUM_ROWS = 10_000

data = [papers_ids[NUM_ROWS*4:NUM_ROWS*5], embeddings[NUM_ROWS*4:NUM_ROWS*5], titles[NUM_ROWS*4:NUM_ROWS*5], abstracts[NUM_ROWS*4:NUM_ROWS*5]]
papers_collection = Collection("research_papers")
papers_collection.load()
papers_collection.insert(data)
papers_collection.flush()
print(papers_collection.num_entities)

In [ ]:
# Search test
query = "machine learning"
query_embedding = model.encode(query)
query_E = query_embedding.tolist()
top_k = 15

search_params = {"metric_type": "IP"}

papers_collection = Collection("research_papers")
papers_collection.load()


results = papers_collection.search(
    data = [query_E],
    anns_field = "embedding",
    param = search_params,
    output_fields=['paper_id', 'title', 'abstract'],
    limit = top_k
)

# print(results[0].distances)


print(results)

In [ ]:
results = results[0]
ret = []

for match in results:
    data = {}
    data['id'] = match.id
    for key in match.entity.fields:
        data[key] = match.entity.get(key)
    ret.append(data)
for r in ret:
  print(r)